#### Step 1: Importing Libraries

In [2]:
import json
import pandas as pd
import glob

#### Step 2: Data Loading

In [7]:
# Step 1: Get all JSON file names from the folder
files = glob.glob("*.json") # change this to your folder path

print("Found files:", files)

Found files: ['energy.json', 'social_and_economic.json', 'electricity.json']


In [10]:
# Step 2: Read and combine them all
dataframes = []

for file in files:
    with open(file, "r") as f:
        data = json.load(f)
        df = pd.DataFrame(data)
        dataframes.append(df)

print(dataframes)

[      id      name      score                unit      region_name  \
0     DZ   Algeria   1.122612  Millions of people     North Africa   
1     DZ   Algeria   0.947765  Millions of people     North Africa   
2     DZ   Algeria   0.831871  Millions of people     North Africa   
3     DZ   Algeria   0.712877  Millions of people     North Africa   
4     DZ   Algeria   0.623791  Millions of people     North Africa   
...   ..       ...        ...                 ...              ...   
3721  ZW  Zimbabwe  30.299999     % of population  Southern Africa   
3722  ZW  Zimbabwe  30.299999     % of population  Southern Africa   
3723  ZW  Zimbabwe       30.5     % of population  Southern Africa   
3724  ZW  Zimbabwe       30.5     % of population  Southern Africa   
3725  ZW  Zimbabwe  30.799999     % of population  Southern Africa   

     indicator_topic   indicator_source  \
0             Access  Tracking SDG7/WBG   
1             Access  Tracking SDG7/WBG   
2             Access  Trackin

In [11]:
# Step 3: Combine all into one DataFrame
combined_df = pd.concat(dataframes, ignore_index=True)

print("Combined shape:", combined_df.shape)
print(combined_df.head())


Combined shape: (50922, 11)
   id     name     score                unit   region_name indicator_topic  \
0  DZ  Algeria  1.122612  Millions of people  North Africa          Access   
1  DZ  Algeria  0.947765  Millions of people  North Africa          Access   
2  DZ  Algeria  0.831871  Millions of people  North Africa          Access   
3  DZ  Algeria  0.712877  Millions of people  North Africa          Access   
4  DZ  Algeria  0.623791  Millions of people  North Africa          Access   

    indicator_source                                     indicator_name  \
0  Tracking SDG7/WBG  Energy: Population without access to clean coo...   
1  Tracking SDG7/WBG  Energy: Population without access to clean coo...   
2  Tracking SDG7/WBG  Energy: Population without access to clean coo...   
3  Tracking SDG7/WBG  Energy: Population without access to clean coo...   
4  Tracking SDG7/WBG  Energy: Population without access to clean coo...   

  indicator_group  year                   url  
0   

#### Step 3: Data Cleaning & Formatting

In [12]:
# Step 1: Rename columns
df = combined_df.rename(columns={
    "id": "country_serial",
    "name": "country",
    "indicator_name": "metric",
    "unit": "unit",
    "indicator_group": "sector",
    "indicator_topic": "sub_sector",
    "indicator_source": "source",
    "url": "source_link",
    "score": "value"
})

df.head()

,country_serial,country,value,unit,region_name,sub_sector,source,metric,sector,year,source_link
0,DZ,Algeria,1.122612,Millions of people,North Africa,Access,Tracking SDG7/WBG,Energy: Population without access to clean coo...,Energy,2000,/aep/country/algeria
1,DZ,Algeria,0.947765,Millions of people,North Africa,Access,Tracking SDG7/WBG,Energy: Population without access to clean coo...,Energy,2001,/aep/country/algeria
2,DZ,Algeria,0.831871,Millions of people,North Africa,Access,Tracking SDG7/WBG,Energy: Population without access to clean coo...,Energy,2002,/aep/country/algeria
3,DZ,Algeria,0.712877,Millions of people,North Africa,Access,Tracking SDG7/WBG,Energy: Population without access to clean coo...,Energy,2003,/aep/country/algeria
4,DZ,Algeria,0.623791,Millions of people,North Africa,Access,Tracking SDG7/WBG,Energy: Population without access to clean coo...,Energy,2004,/aep/country/algeria


In [6]:
# Step 2: Convert value column to numeric
df["value"] = pd.to_numeric(df["value"], errors="coerce")

- The above code cell “Convert every entry in the ‘value’ column to a number. e.g. Some values might appear as "36.5" instead of 36.5

- If something cannot be converted (like ‘N/A’ or text), replace it with NaN (which means missing value).

- If we try to perform calculations or reshape this data while it’s still text, pandas will throw an error because it can’t compute mathematical operations on strings.

- `errors="coerce"` tells pandas: “If you can’t convert something to a number, turn it into NaN (missing value), don’t crash.”

In [13]:
# Step 3: Pivot data (years → columns)
pivot_df = df.pivot_table(
    index=[
        "country", "country_serial", "metric", "unit",
        "sector", "sub_sector", "source_link", "source"
    ],
    columns="year",
    values="value",
    aggfunc="first"   # safer than 'mean'
).reset_index()

In [17]:
# Step 4: Flatten the columns - removes the year column
pivot_df.columns.name = None

In [18]:
# Step 5: Save to CSV
pivot_df.to_csv("africa_energy_data.csv", index=False)

pivot_df.head()

,country,country_serial,metric,unit,sector,sub_sector,source_link,source,2000,2001,...,2013,2014,2015,2016,2017,2018,2019,2020,2021,2022
0,Algeria,DZ,Electricity export (GWh),GWh,Electricity,Supply,/aep/country/algeria,AFREC Database,319,196,...,384,877,641,507,918.764526,833.876404,975.834595,1128.864014,1529.344971,NULL
1,Algeria,DZ,Electricity final consumption (GWh),GWh,Electricity,Supply,/aep/country/algeria,AFREC Database,18592,19664,...,40188,45751,47956.96875,52288,56376.101562,58152.601562,59053.726562,60044.328125,62502.121094,NULL
2,Algeria,DZ,Electricity final consumption per capita (KWh),KWh per capita,Electricity,Supply,/aep/country/algeria,AFREC Database,596.209656,622.433105,...,1048.239624,1169.704102,1202.787354,1287.689941,1364.439453,1384.320435,1383.671143,1385.640747,1421.494507,NULL
3,Algeria,DZ,Electricity generated from biofuels and waste ...,GWh,Electricity,Supply,/aep/country/algeria,AFREC Database,0,0,...,0,0,0,0,0,0,0,0,0,NULL
4,Algeria,DZ,Electricity generated from fossil fuels (GWh),GWh,Electricity,Supply,/aep/country/algeria,AFREC Database,25358,26556,...,59560,63988,68576,70663,75381.96875,75879.992188,79441.46875,83299.226562,84709.507812,NULL


In [19]:
pivot_df.shape

(1944, 31)

#### Step 4: Data Storage

In [8]:
!pip install pymongo

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.7/1.7 MB 3.5 MB/s eta 0:00:00m eta 0:00:010:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 331.1/331.1 kB 5.9 MB/s eta 0:00:000m eta 0:00:01


In [20]:
from pymongo import MongoClient

# Step 1: Ensure year columns are strings - In MongoDB, field names (the keys) must always be strings.
pivot_df.columns = pivot_df.columns.map(str)

In [ ]:
# Step 2: Replace NaN with None (MongoDB does not accept NaN)
pivot_df = pivot_df.where(pd.notnull(pivot_df), None)

- MongoDB does not accept NaN values

- When you export from pandas to MongoDB, pandas uses NaN (Not a Number) to represent missing values.

- But in MongoDB (and in JSON), there is no such thing as NaN.

- The valid equivalent for “missing” is null, which corresponds to None in Python.

In [ ]:
# Step 3: Convert DataFrame to list of dicts
records = pivot_df.to_dict(orient="records")
print(records)

- This code cell above your entire pandas DataFrame (pivot_df) into a list of Python dictionaries, where:

    - each row in the DataFrame becomes one dictionary (document)

    - each column name becomes a key in that dictionary

- MongoDB doesn’t understand pandas DataFrames directly. MongoDB stores data as JSON-like documents

- It expects data in a structure like:

`{"key": "value", "key2": "value2"}`

In [22]:
# Step 4: Connect to MongoDB
client = MongoClient("mongodb://localhost:27017/")
db = client["africa_energy_db"]
collection = db["energy_data"]

In [23]:
# Step 5: Insert data
collection.insert_many(records)

InsertManyResult([ObjectId('68f0732a96d2d76cd460d0dc'), ObjectId('68f0732a96d2d76cd460d0dd'), ObjectId('68f0732a96d2d76cd460d0de'), ObjectId('68f0732a96d2d76cd460d0df'), ObjectId('68f0732a96d2d76cd460d0e0'), ObjectId('68f0732a96d2d76cd460d0e1'), ObjectId('68f0732a96d2d76cd460d0e2'), ObjectId('68f0732a96d2d76cd460d0e3'), ObjectId('68f0732a96d2d76cd460d0e4'), ObjectId('68f0732a96d2d76cd460d0e5'), ObjectId('68f0732a96d2d76cd460d0e6'), ObjectId('68f0732a96d2d76cd460d0e7'), ObjectId('68f0732a96d2d76cd460d0e8'), ObjectId('68f0732a96d2d76cd460d0e9'), ObjectId('68f0732a96d2d76cd460d0ea'), ObjectId('68f0732a96d2d76cd460d0eb'), ObjectId('68f0732a96d2d76cd460d0ec'), ObjectId('68f0732a96d2d76cd460d0ed'), ObjectId('68f0732a96d2d76cd460d0ee'), ObjectId('68f0732a96d2d76cd460d0ef'), ObjectId('68f0732a96d2d76cd460d0f0'), ObjectId('68f0732a96d2d76cd460d0f1'), ObjectId('68f0732a96d2d76cd460d0f2'), ObjectId('68f0732a96d2d76cd460d0f3'), ObjectId('68f0732a96d2d76cd460d0f4'), ObjectId('68f0732a96d2d76cd460d0

In [ ]:
# Step 6: Verify
print("Inserted:", collection.count_documents({}), "documents")
print(collection.find_one())

Inserted: 1943 documents
{'_id': ObjectId('68ef1e0f36d3b506fcf02f9a'), 'country': 'Algeria', 'country_serial': 'DZ', 'metric': 'Electricity export (GWh)', 'unit': 'GWh', 'sector': 'Electricity', 'sub_sector': 'Supply', 'source_link': '/aep/country/algeria', 'source': 'AFREC Database', '2000': 319.0, '2001': 196.0, '2002': 259.0, '2003': 212.0, '2004': 197.0, '2005': 275.0, '2006': 298.0, '2007': 273.0, '2008': 323.0, '2009': 362.0, '2010': 803.0, '2011': 799.0, '2012': 985.0, '2013': 384.0, '2014': 877.0, '2015': 641.0, '2016': 507.0, '2017': 918.764526367188, '2018': 833.876403808594, '2019': 975.834594726563, '2020': 1128.86401367188, '2021': 1529.34497070313, '2022': nan}


Step 5: Data Validation

In [24]:
# Step 1: Load your CSV
df = pd.read_csv("africa_energy_data.csv")
print("Shape:", df.shape)
print(df.head())

Shape: (1944, 31)
   country country_serial                                             metric  \
0  Algeria             DZ                           Electricity export (GWh)   
1  Algeria             DZ                Electricity final consumption (GWh)   
2  Algeria             DZ     Electricity final consumption per capita (KWh)   
3  Algeria             DZ  Electricity generated from biofuels and waste ...   
4  Algeria             DZ      Electricity generated from fossil fuels (GWh)   

             unit       sector sub_sector           source_link  \
0             GWh  Electricity     Supply  /aep/country/algeria   
1             GWh  Electricity     Supply  /aep/country/algeria   
2  KWh per capita  Electricity     Supply  /aep/country/algeria   
3             GWh  Electricity     Supply  /aep/country/algeria   
4             GWh  Electricity     Supply  /aep/country/algeria   

           source          2000          2001  ...          2013  \
0  AFREC Database    319.00000

In [27]:
# Step 2: Check for missing values per column
missing_summary = df.isnull().sum()
print("Missing values per column:")
print(missing_summary[missing_summary > 0])

Missing values per column:
country_serial     36
2000                9
2001                9
2002                9
2003                9
2004                9
2005                9
2006                9
2007                9
2008                9
2009                9
2010                9
2011                8
2012                1
2013                1
2014                1
2015                1
2016                1
2017                1
2018                1
2019                2
2020                1
2021                1
2022              785
dtype: int64


In [29]:
# Step 3: Check rows with missing year values
year_cols = [col for col in df.columns if col.isdigit()]
print("Detected year columns:", year_cols, "\n")

missing_years = df[df[year_cols].isnull().any(axis=1)]
print(f"Found {len(missing_years)} rows with missing year values.")

Detected year columns: ['2000', '2001', '2002', '2003', '2004', '2005', '2006', '2007', '2008', '2009', '2010', '2011', '2012', '2013', '2014', '2015', '2016', '2017', '2018', '2019', '2020', '2021', '2022'] 

Found 792 rows with missing year values.


In [30]:
# Step 4: Check for missing or blank metric names
missing_metrics = df[df["metric"].isnull() | (df["metric"] == "")]
print(f"Rows missing metric names: {len(missing_metrics)}")

Rows missing metric names: 0


In [31]:
# Step 5: Check for unit consistency
print("Unique units:")
print(df["unit"].unique(), "\n")


unit_check = df.groupby("metric")["unit"].nunique().reset_index()
inconsistent_units = unit_check[unit_check["unit"] > 1]
print(f"Metrics with inconsistent units: {len(inconsistent_units)}")
if len(inconsistent_units) > 0:
    print(inconsistent_units, "\n")

Unique units:
['GWh' 'KWh per capita' 'MW' 'MJ/USD PPP 2017' 'Millions of people'
 'Current US$'] 

Metrics with inconsistent units: 0


In [33]:
# Step 6: Check country naming consistency
countries = sorted(df["country"].unique())
print(f"Total unique countries: {len(countries)}")
print("Sample countries:", countries[:10])

Total unique countries: 54
Sample countries: ['Algeria', 'Angola', 'Benin', 'Botswana', 'Burkina Faso', 'Burundi', 'Cameroon', 'Cape Verde', 'Central African Republic', 'Chad']


In [ ]:
# Step 7: Final validation summary
summary = {
    "Total rows": len(df),
    "Rows with missing year data": len(missing_years),
    "Rows missing metric names": len(missing_metrics),
    "Metrics with inconsistent units": len(inconsistent_units),
    "Unique countries": len(countries)
}

print("Data Validation Summary:")
for key, value in summary.items():
    print(f"{key}: {value}")

Data Validation Summary:
Total rows: 1944
Rows with missing year data: 792
Rows missing metric names: 0
Metrics with inconsistent units: 0
Unique countries: 54
